In [29]:
import cv2

In [30]:
cap = cv2.VideoCapture(0)

In [31]:
status, photo = cap.read()

In [32]:
cv2.imshow("Cropped Face with Rectangle" , photo)
cv2.waitKey()
cv2.destroyAllWindows()

In [33]:
facemodel = cv2.CascadeClassifier("myhaarcascade_frontalface_default.xml")

In [34]:
len( facemodel.detectMultiScale(photo) )

1

In [35]:
facemodel.detectMultiScale(photo)

array([[217, 103, 214, 214]])

In [36]:
while True:
    status, photo = cap.read()

    myfacecoord = facemodel.detectMultiScale(photo)
#print(myfacecoord)

    if len(myfacecoord) == 1:
        x1 = myfacecoord[0][0]
        y1 = myfacecoord[0][1]
        x2 = myfacecoord[0][0] + myfacecoord[0][2]
        y2 = myfacecoord[0][1] + myfacecoord[0][3]

        cv2.rectangle(photo , (x1,y1) , (x2,y2) , [0,255,0], 5  )

    cv2.imshow("hi" , photo)
    if cv2.waitKey(10) == 13:
        break

cap.release()        
cv2.destroyAllWindows()

In [45]:


def blur_background(image, faces):
   
    result_image = image.copy()

    
    result_image = cv2.GaussianBlur(result_image, (21, 21), 0)

    for (x, y, w, h) in faces:
        
        face_roi = result_image[y:y + h, x:x + w]

      
        face_roi = cv2.GaussianBlur(face_roi, (0, 0), 30)

       
        result_image[y:y + h, x:x + w] = face_roi

    return result_image





gray = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)


faces = facemodel.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))


result = blur_background(photo, faces)


cv2.imshow('Result', result)
 if cv2.waitKey(10) == 13:
        break

cap.release()        
cv2.destroyAllWindows()


In [46]:
import cv2
import math





KNOWN_FACE_WIDTH = 14  
FOCAL_LENGTH = 500 

def estimate_distance(face_width_pixels):
    
    distance = (KNOWN_FACE_WIDTH * FOCAL_LENGTH) / face_width_pixels
    return distance


cap = cv2.VideoCapture(0)

while True:
  
    ret, frame = cap.read()

    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    
    faces = facemodel.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        
        distance = estimate_distance(w)

        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f"Distance: {distance:.2f} cm", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

   
    cv2.imshow('Face Distance Estimation', frame)

    if cv2.waitKey(10) == 13:
        break

cap.release()        
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np



eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')


sunglasses_img = cv2.imread("glasses.png", cv2.IMREAD_UNCHANGED)

def overlay_sunglasses(frame, x, y, w, h):
   
    roi_gray = frame[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray)

    if len(eyes) >= 2:

        eyes = sorted(eyes, key=lambda x: x[0])

        
        eye1_x, eye1_y, eye1_w, eye1_h = eyes[0]
        eye2_x, eye2_y, eye2_w, eye2_h = eyes[1]
        eye1_center = (x + eye1_x + eye1_w // 2, y + eye1_y + eye1_h // 2)
        eye2_center = (x + eye2_x + eye2_w // 2, y + eye2_y + eye2_h // 2)

        
        eyes_distance = eye2_center[0] - eye1_center[0]

       
        sunglasses_width = int(1.5 * eyes_distance)
        sunglasses_height = int(sunglasses_width * sunglasses_img.shape[0] / sunglasses_img.shape[1])

       
        sunglasses_x = eye1_center[0] - int(0.25 * eyes_distance)
        sunglasses_y = eye1_center[1] - int(0.4 * sunglasses_height)

       
        sunglasses_resized = cv2.resize(sunglasses_img, (sunglasses_width, sunglasses_height))

       
        mask = sunglasses_resized[:, :, 3]

        
        mask_inv = cv2.bitwise_not(mask)

        
        sunglasses_colored = sunglasses_resized[:, :, 0:3]

        
        roi = frame[sunglasses_y:sunglasses_y + sunglasses_height, sunglasses_x:sunglasses_x + sunglasses_width]
        roi_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
        roi_fg = cv2.bitwise_and(sunglasses_colored, sunglasses_colored, mask=mask)

       
        dst = cv2.add(roi_bg, roi_fg)
        frame[sunglasses_y:sunglasses_y + sunglasses_height, sunglasses_x:sunglasses_x + sunglasses_width] = dst


cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    
    faces = facemodel.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        overlay_sunglasses(frame, x, y, w, h)

   
    cv2.imshow('Face with Sunglasses', frame)

   if cv2.waitKey(10) == 13:
        break

cap.release()        
cv2.destroyAllWindows()
